In [5]:
import cv2
import numpy as np
import pytesseract
from PIL import Image

# openCV를 통해 한글 파일명으로 저장하는 것을 습득하지 못하여 다소 번거로운 과정이 포함되어있음.

In [12]:
# 마구마구 게임 내 도감 스크린샷을 불러온 뒤 좌표를 이용하여 카드의 위치, 크기에 대해 입력 (단순 입력)
x_start = [509, 633, 757, 881, 1005, 1129, 1253]
y_start = [205, 373, 541, 709]
img = Image.open('2014.png')
char = 1
area2 = (20, 141, 100, 160)

# for 문을 통해 최대 28장의 카드를 모두 잘라내어 카드 선수 연도 및 이름으로 저장을 시도
for i in x_start:
    for j in y_start:
        try:
            area = (i,j,i+118,j+162)
            cropped_img = img.crop(area)
            dst = cropped_img.crop(area2)
            result = pytesseract.image_to_string(dst, lang='kor')
            
            # 1차시도 // tesseract OCR을 통해 카드에 쓰여진 연도와 선수 이름을 인식 시도
            if result != '':
                name = '%s.png' % result
                cropped_img.save(name)
                
            else:
                name = '14 '+str(char)+'.png'
                dst.save(name)
                cv2_img = cv2.imread(name)
                gray = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2GRAY)
                result = pytesseract.image_to_string(gray, lang='kor')
                
                # 2차시도 // 인식률을 높이기 위해 이미지를 흑백화
                if result != '':
                    name = '%s.png' % result
                    cropped_img.save(name)
                    
                else:
                    gray.save(name)
                    cv2_img = cv2.imread(name)

                    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
                    imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
                    imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)
                    imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
                    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)
                    result = pytesseract.image_to_string(gray, lang='kor')
                    
                    # 3차시도 // 인식률을 더 높임
                    if result != '':
                        name = '%s.png' % result
                        cropped_img.save(name)
                        
                    else:
                        cropped_img.save(name)
                        char = char + 1
                        
        # 인식을 모두 실패하거나 오류가 발생할 경우, 연도로만 자동 저장 후 수동으로 이름을 설정
        except:
            cropped_img.save('14 '+str(char)+'.png')
            char = char + 1
